# Translation

In [0]:
import os
from tqdm.auto import tqdm

In [0]:
data = []

PATH = './icelandic_parsed'
for fname in sorted(os.listdir(PATH)):
    with open(os.path.join(PATH, fname)) as f:
        text = f.read().split('\n\n')
    data.append([fname.strip('tx.'), text])

In [0]:
s = 0
for row in data:
    s += sum(len(line) for line in row[1])
s

8323013

In [0]:
import json

## Yandex

In [0]:
import warnings
warnings.filterwarnings("ignore")
import requests

In [0]:
headers = {
    'User-Agent': ('Mozilla/5.0 (Windows NT 6.0; rv:14.0) Gecko/20100101 '
                   'Firefox/14.0.1'),
    'Accept':
    'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
    'Accept-Language':
    'ru-ru,ru;q=0.8,en-us;q=0.5,en;q=0.3',
    'Accept-Encoding':
    'gzip, deflate',
    'Connection':
    'keep-alive',
    'DNT':
    '1'
}
URL = "https://translate.yandex.net/api/v1.5/tr.json/translate"

In [0]:
def translate(text, lang="en"):
    params = {
    "key": KEY,
    "text": text,
    "lang": f"is-{lang}"
    }
    response = requests.get(URL, params=params, headers=headers)
    return response.json()

In [0]:
translate("halló", "en")

{'code': 200, 'lang': 'is-en', 'text': ['hello']}

In [0]:
## Database

In [0]:
import sqlite3

db = sqlite3.connect('translations.db')
cur = db.cursor()

In [0]:
cur.execute("""
CREATE TABLE docs (
id INTEGER PRIMARY KEY AUTOINCREMENT,
name TEXT
)
""")

cur.execute("""
CREATE TABLE sentences (
id INTEGER PRIMARY KEY AUTOINCREMENT,
id_text INT,
idx INT,
text TEXT,
udpipe TEXT
)
""")

cur.execute("""
CREATE TABLE yandex_en (
id INTEGER PRIMARY KEY,
text TEXT
udpipe TEXT
)
""")

db.commit()

Сохранить тексты в базу

In [0]:
texts = []
for idx, row in enumerate(data):
    texts.append((idx+1, row[0]))
cur.executemany('INSERT INTO docs VALUES (?, ?)', texts)
db.commit()

In [0]:
sentences = []

for idx, row in enumerate(data):
    for ids, sentence in enumerate(row[1]):
        sentences.append((idx+1, ids+1, sentence))
cur.executemany('INSERT INTO sentences (id_text, idx, text) VALUES (?, ?, ?)', sentences)
db.commit()

In [0]:
cur.execute('SELECT name, id FROM docs')
text_indices = {key: value for key, value in cur.fetchall()}

Это то, что переведено другим способом

In [0]:
for fname in os.listdir('./yandex_en/'):
    try:
        data = json.load(open(os.path.join('./yandex_en/', fname)))
        ind = text_indices[data['fname']]
        start = cur.execute('SELECT min(id) FROM sentences WHERE id_text = ?;', (ind,)).fetchone()[0]
        sentences = data['lines']
        result = []
        for idx, sent in enumerate(sentences):
            if sent[1] is not None:
                result.append((start+idx, sent[1][0]))
        cur.executemany('INSERT INTO yandex_en (id, text) VALUES (?, ?)', result)
        db.commit()
    except Exception as e:
        print(e)
        print(fname)

### Yandex

In [0]:
def get_sentences(lang, limit=100):
    cur.execute(f"""
    SELECT id, text FROM sentences WHERE id NOT IN (
        SELECT id FROM yandex_{lang}
    )
    LIMIT ?
    """, (limit,))
    data = cur.fetchall()
    #print(data)
    result = []
    summ = 0
    for row in data:
        try:
            sentence = translate(row[1], lang).get('text')
            assert len(sentence) == 1
            sentence = sentence[0]
            result.append((row[0], sentence,))
        except Exception as e:
            pass
        summ += len(row[1])
    if len(result) != limit:
        print(len(result), end=' ')
    cur.executemany(f'INSERT INTO yandex_{lang} (id, text) VALUES (?, ?)', result)
    db.commit()
    return summ

In [0]:
S = 0
for _ in tqdm(range(100)):
    get_sentences('en', limit=100)

In [0]:
KEYS = [

]

In [0]:
for key, s in KEYS:
    S = s
    KEY = key
    while S < 980000:
        S += get_sentences('sv', limit=100)
    print(S)

In [0]:
SELECT count(id)*1.0/101359.0 FROM yandex_en;

In [0]:
sv

### Google

In [0]:
from googletrans import Translator
translator = Translator()

In [0]:
class Proxy:
    def __init__(self, fname):
        with open(fname) as f:
            self.proxies = [s.strip() for s in f.readlines()]
    
    def new(self):
        idx = len(self.proxies)-1
        del self.proxies[idx]
        current = self.proxies[-1]
        return {'http': current, 'https': current}
    
    def current(self):
        current = self.proxies[-1]
        return {'http': current, 'https': current}

In [0]:
PROXY = Proxy('proxies.txt')

In [0]:
PROXY.current()

{'http': '114.239.29.244:9999', 'https': '114.239.29.244:9999'}

In [0]:
def get_sentences(lang, limit=100):
    cur.execute(f"""
    SELECT id, text FROM sentences WHERE id NOT IN (
        SELECT id FROM google_{lang}
    )
    LIMIT ?
    """, (limit,))
    data = cur.fetchall()
    result = []
    translator = Translator(proxies=PROXY.current(), timeout=2)
    for row in tqdm(data, leave=False):
        try:
            sentence = translator.translate(row[1], src='is', dest=lang)
            sentence = sentence.text
            result.append((row[0], sentence,))
        except Exception as e:
            translator = Translator(proxies=PROXY.new(), timeout=2)
    if len(result) != limit:
        print(len(result), end=' ')
    cur.executemany(f'INSERT INTO google_{lang} (id, text) VALUES (?, ?)', result)
    db.commit()

In [0]:
from IPython.display import clear_output

In [0]:
while True:
    try:
        PROXY = Proxy('proxies.txt')
        for _ in tqdm(range(1000)):
            get_sentences('da', limit=100)
    except IndexError:
        clear_output()

In [0]:
translator = Translator()
res = translator.translate('Ekki var honum fjárhagur sinn hægur , átti lendur miklar en minna lausafé .', src='is', dest='en')